In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import time
import os
import torch.nn.functional as F
from tqdm import tqdm


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True
1


In [3]:
learning_rate = 0.001
num_epochs = 2
num_workers = 2
batch_size = 100

In [4]:
# Load CIFAR-100 data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Define the Teacher model
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.fc1 = nn.Linear(128*5*5, 256)
        self.fc2 = nn.Linear(256, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 128*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the ModifiableStudent class
class ModifiableStudent(nn.Module):
    def __init__(self, num_classes=100):
        super(ModifiableStudent, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*14*14, 120)
        self.fc2 = nn.Linear(120, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [6]:
# def fairness_constraint(outputs, sensitive_attrs):
#     # This is a simplified fairness constraint for demonstration purposes.
#     group0_indices = (sensitive_attrs < 5).nonzero(as_tuple=True)
#     group1_indices = (sensitive_attrs >= 5).nonzero(as_tuple=True)
#     positive_rate_group0 = (outputs[group0_indices].max(1)[1] >= 5).float().mean()
#     positive_rate_group1 = (outputs[group1_indices].max(1)[1] >= 5).float().mean()
#     return torch.abs(positive_rate_group0 - positive_rate_group1)

# def adversarial_train_model(model, dataloader, criterion, optimizer, epoch):
#     model.train()
#     running_loss = 0.0
#     progress_bar = tqdm(enumerate(dataloader, 0), total=len(dataloader), unit="batch")
#     for i, data in progress_bar:
#         inputs, labels = data
#         sensitive_attrs = labels  # Assuming label itself as a sensitive attribute
#         optimizer.zero_grad()
#         outputs = model(inputs.to(device))
#         loss = criterion(outputs, labels.to(device))
#         fairness_penalty = fairness_constraint(outputs, sensitive_attrs.to(device))
#         combined_loss = loss + fairness_penalty  # Combine original loss and fairness penalty
#         combined_loss.backward()
#         optimizer.step()
#         running_loss += combined_loss.item()
#         progress_bar.set_description(f"Epoch {epoch} Loss: {running_loss/(i+1):.4f}")
#     return running_loss / len(dataloader)

In [7]:
# Define the fair_metric function from the RELIANT code
def fair_metric(pred, labels, sens):
    idx_s0 = sens == 0
    idx_s1 = sens == 1
    idx_s0_y1 = idx_s0 & (labels == 1)
    idx_s1_y1 = idx_s1 & (labels == 1)

    if idx_s0.sum().item() == 0 or idx_s1.sum().item() == 0 or idx_s0_y1.sum().item() == 0 or idx_s1_y1.sum().item() == 0:
        return torch.tensor(0.0), torch.tensor(0.0)  # Return zeros

    # print(idx_s0.sum().item(), idx_s1.sum().item(), idx_s0_y1.sum().item(), idx_s1_y1.sum().item())
    # smoothing term added
    parity = torch.abs((pred[idx_s0].sum() / (idx_s0.sum().item() + 1e-8)) - (pred[idx_s1].sum() / (idx_s1.sum().item() + 1e-8)))
    equality = torch.abs((pred[idx_s0_y1].sum() / (idx_s0_y1.sum().item() + 1e-8)) - (pred[idx_s1_y1].sum() / (idx_s1_y1.sum().item() + 1e-8)))

    return parity.item(), equality.item()

In [8]:
# Define the approx_loss and approx_loss_eo functions from the RELIANT code
def approx_loss(logits, sens, idx):
    g1 = np.argwhere(sens == 0.0).reshape(-1)
    g2 = np.argwhere(sens == 1.0).reshape(-1)
    idx_set = set(idx.cpu().numpy())
    g1 = np.array(list(set(g1) & idx_set))
    g2 = np.array(list(set(g2) & idx_set))

    if g1.shape[0] == 0 or g2.shape[0] == 0:
        return torch.tensor(0.0) # Return zero
    
    # print(g1.shape[0], g2.shape[0])
    # smoothing term added
    loss = torch.square(approx_func(logits[g1]).sum(axis=0) / (g1.shape[0] + 1e-8) - approx_func(logits[g2]).sum(axis=0) / (g2.shape[0] + 1e-8)).sum()
    return loss


In [9]:
def approx_loss_eo(logits, sens, labels, idx):
    g1 = np.argwhere(sens == 0).reshape(-1)
    g2 = np.argwhere(sens == 1).reshape(-1)
    g = np.argwhere(labels.cpu().numpy() == 1).reshape(-1)
    idx_set = set(idx.cpu().numpy())
    g1 = np.array(list(set(g1) & set(g) & idx_set))
    g2 = np.array(list(set(g2) & set(g) & idx_set))
    
    if g1.shape[0] == 0 or g2.shape[0] == 0:
        return torch.tensor(0.0)  # Return zero 
        
    # print(g1.shape[0], g2.shape[0])
    # smoothing term added
    loss = torch.square(approx_func(logits[g1]).sum(axis=0) / (g1.shape[0] + 1e-8) - approx_func(logits[g2]).sum(axis=0) / (g2.shape[0] + 1e-8)).sum()
    return loss

In [10]:
def approx_func(s):
    x = 2 * s - 1
    result = 1 / 2 + 1 / 2 * x - 1 / 8 / 2 * (5 * x * x * x - 3 * x) + 1 / 16 / 8 * (
                63 * x * x * x * x * x - 70 * x * x * x + 15 * x) - 5 / 128 / 16 * (
                       429 * x * x * x * x * x * x * x - 693 * x * x * x * x * x + 315 * x * x * x - 35 * x) + 7 / 256 / 128 * (
                       12155 * x * x * x * x * x * x * x * x * x - 25740 * x * x * x * x * x * x * x + 18018 * x * x * x * x * x - 4620 * x * x * x + 315 * x)
    # print(s, x, result)
    return result

In [11]:
# Define the logging lists at the beginning of your script or before the training loop
teacher_loss_log, teacher_combined_loss_log, teacher_fairness_penalty_sp_log, teacher_fairness_penalty_eo_log, teacher_fairness_penalty_approx_log, teacher_fairness_penalty_approx_eo_log = [], [], [], [], [], []
student_loss_log, student_combined_loss_log, student_fairness_penalty_sp_log, student_fairness_penalty_eo_log, student_fairness_penalty_approx_log, student_fairness_penalty_approx_eo_log = [], [], [], [], [], []


In [12]:
def adversarial_train_model(model, dataloader, criterion, optimizer, epoch, loss_log, combined_loss_log, fairness_penalty_sp_log, fairness_penalty_eo_log, fairness_penalty_approx_log, fairness_penalty_approx_eo_log):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(enumerate(dataloader, 0), total=len(dataloader), unit="batch")
    for i, data in progress_bar:
        inputs, labels = data
        sensitive_attrs = labels  # Assuming label itself as a sensitive attribute
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        # Incorporate fairness metrics
        fairness_penalty_sp, fairness_penalty_eo = fair_metric(outputs, labels, sensitive_attrs)
        fairness_penalty_approx = approx_loss(outputs, sensitive_attrs, torch.arange(len(labels)).to(device))
        fairness_penalty_approx_eo = approx_loss_eo(outputs, sensitive_attrs, labels, torch.arange(len(labels)).to(device))
        combined_loss = loss + fairness_penalty_sp + fairness_penalty_eo + fairness_penalty_approx + fairness_penalty_approx_eo

        print(combined_loss.item(), fairness_penalty_sp, fairness_penalty_eo, fairness_penalty_approx.item(), fairness_penalty_approx_eo.item())
        combined_loss.backward()
        optimizer.step()
        running_loss += combined_loss.item()
        progress_bar.set_description(f"Epoch {epoch} Loss: {running_loss/(i+1):.4f}")
        
        # Logging the losses
        loss_log.append(loss.item())
        combined_loss_log.append(combined_loss.item())
        fairness_penalty_sp_log.append(fairness_penalty_sp)
        fairness_penalty_eo_log.append(fairness_penalty_eo)
        fairness_penalty_approx_log.append(fairness_penalty_approx.item())
        fairness_penalty_approx_eo_log.append(fairness_penalty_approx_eo.item())
    return running_loss / len(dataloader)

In [13]:
# Initialize models, optimizers, and loss function
teacher = Teacher().to(device)
student = ModifiableStudent().to(device)
optimizer_teacher = optim.Adam(teacher.parameters(), lr=learning_rate)
optimizer_student = optim.Adam(student.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [14]:
# Training Loop
for epoch in range(num_epochs):
    adversarial_train_model(teacher, trainloader, criterion, optimizer_teacher, epoch, teacher_loss_log, teacher_combined_loss_log, teacher_fairness_penalty_sp_log, teacher_fairness_penalty_eo_log, teacher_fairness_penalty_approx_log, teacher_fairness_penalty_approx_eo_log)
    adversarial_train_model(student, trainloader, criterion, optimizer_student, epoch, student_loss_log, student_combined_loss_log, student_fairness_penalty_sp_log, student_fairness_penalty_eo_log, student_fairness_penalty_approx_log, student_fairness_penalty_approx_eo_log)


Epoch 0 Loss: 4.5995:   0%|                  | 1/500 [00:00<03:09,  2.64batch/s]

4.611886501312256 tensor(0.) tensor(0.) 0.0 0.0
4.620633125305176 tensor(0.) tensor(0.) 0.0 0.0
4.618854522705078 tensor(0.) tensor(0.) 0.0 0.0
4.6222147941589355 tensor(0.) tensor(0.) 0.0 0.0
4.5675129890441895 tensor(0.) tensor(0.) 0.0 0.0
4.594176769256592 tensor(0.) tensor(0.) 0.0 0.0
4.561077117919922 tensor(0.) tensor(0.) 0.0 0.0


Epoch 0 Loss: 4.5171:   3%|▌                | 17/500 [00:00<00:12, 38.97batch/s]

4.5786333084106445 tensor(0.) tensor(0.) 0.0 0.0
4.587055206298828 tensor(0.) tensor(0.) 0.0 0.0
4.532139778137207 tensor(0.) tensor(0.) 0.0 0.0
4.500350475311279 tensor(0.) tensor(0.) 0.0 0.0
4.506548881530762 tensor(0.) tensor(0.) 0.0 0.0
4.5078301429748535 tensor(0.) tensor(0.) 0.0 0.0
4.541199207305908 tensor(0.) tensor(0.) 0.0 0.0
4.489974498748779 tensor(0.) tensor(0.) 0.0 0.0
4.492957592010498 tensor(0.) tensor(0.) 0.0 0.0
4.469582557678223 tensor(0.) tensor(0.) 0.0 0.0
4.429223537445068 tensor(0.) tensor(0.) 0.0 0.0
4.411251544952393 tensor(0.) tensor(0.) 0.0 0.0
4.473464488983154 tensor(0.) tensor(0.) 0.0 0.0
4.391859531402588 tensor(0.) tensor(0.) 0.0 0.0
4.266929626464844 tensor(0.) tensor(0.) 0.0 0.0


Epoch 0 Loss: 4.4214:   6%|█                | 32/500 [00:00<00:08, 54.05batch/s]

...

Epoch 1 Loss: 2.9805:  99%|███████████████▊| 495/500 [00:06<00:00, 71.60batch/s]

2.958977460861206 tensor(0.) tensor(0.) 0.0 0.0
3.095050573348999 tensor(0.) tensor(0.) 0.0 0.0
3.00820255279541 tensor(0.) tensor(0.) 0.0 0.0
2.863396406173706 tensor(0.) tensor(0.) 0.0 0.0
2.9008283615112305 tensor(0.) tensor(0.) 0.0 0.0
3.1650779247283936 tensor(0.) tensor(0.) 0.0 0.0
3.0341506004333496 tensor(0.) tensor(0.) 0.0 0.0
2.750601291656494 tensor(0.) tensor(0.) 0.0 0.0
2.7961461544036865 tensor(0.) tensor(0.) 0.0 0.0
2.641220808029175 tensor(0.) tensor(0.) 0.0 0.0
2.679739475250244 tensor(0.) tensor(0.) 0.0 0.0
3.0127458572387695 tensor(0.) tensor(0.) 0.0 0.0


Epoch 1 Loss: 2.9790:  99%|███████████████▊| 495/500 [00:06<00:00, 71.60batch/s]

2.9895544052124023 tensor(0.) tensor(0.) 0.0 0.0
2.8195157051086426 tensor(0.) tensor(0.) 0.0 0.0
2.665419340133667 tensor(0.) tensor(0.) 0.0 0.0
2.7094738483428955 tensor(0.) tensor(0.) 0.0 0.0


Epoch 1 Loss: 2.9790: 100%|████████████████| 500/500 [00:07<00:00, 71.22batch/s]


In [15]:
# def plot_loss_and_combined_loss(batch_range, loss_log, combined_loss_log, title):
#     plt.figure(figsize=(10, 6))
#     plt.plot(batch_range, loss_log, label='Loss')
#     plt.plot(batch_range, combined_loss_log, label='Total Loss')
#     plt.title(title)
#     plt.xlabel('Batches')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# def plot_fairness_metrics(batch_range, fairness_penalty_sp_log, fairness_penalty_eo_log, fairness_penalty_approx_log, fairness_penalty_approx_eo_log, title):
#     plt.figure(figsize=(10, 6))
#     plt.plot(batch_range, fairness_penalty_sp_log, label='Fairness Penalty SP')
#     plt.plot(batch_range, fairness_penalty_eo_log, label='Fairness Penalty EO')
#     plt.plot(batch_range, fairness_penalty_approx_log, label='Fairness Penalty Approx')
#     plt.plot(batch_range, fairness_penalty_approx_eo_log, label='Fairness Penalty Approx EO')
#     plt.title(title)
#     plt.xlabel('Batches')
#     plt.ylabel('Fairness Penalties')
#     plt.legend()
#     plt.show()

# # 500 batches per epoch and 2 epochs
# total_batches = 500 * num_epochs
# batch_range = np.arange(total_batches)

# # For Teacher
# plot_loss_and_combined_loss(batch_range, teacher_loss_log, teacher_combined_loss_log, 'Teacher Loss and Total Loss')
# plot_fairness_metrics(batch_range, teacher_fairness_penalty_sp_log, teacher_fairness_penalty_eo_log, teacher_fairness_penalty_approx_log, teacher_fairness_penalty_approx_eo_log, 'Teacher Fairness Penalties')

# # For Student
# plot_loss_and_combined_loss(batch_range, student_loss_log, student_combined_loss_log, 'Student Loss and Total Loss')
# plot_fairness_metrics(batch_range, student_fairness_penalty_sp_log, student_fairness_penalty_eo_log, student_fairness_penalty_approx_log, student_fairness_penalty_approx_eo_log, 'Student Fairness Penalties')

In [16]:
# def get_model_size(model):
#     torch.save(model.state_dict(), "temp.p")
#     model_size = os.path.getsize("temp.p")
#     os.remove('temp.p')
#     return model_size
    
# def get_num_parameters(model):
#     return sum(p.numel() for p in model.parameters())

# teacher_model_size = get_model_size(teacher)
# student_model_size = get_model_size(student)
# teacher_num_parameters = get_num_parameters(teacher)
# student_num_parameters = get_num_parameters(student)

In [17]:
# Evaluation and Metrics
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    start_time = time.time()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
    end_time = time.time()
    inference_time = end_time - start_time
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return inference_time, precision, recall, accuracy, f1


In [18]:
# def plot_loss_components(epoch_range, loss_log, combined_loss_log, title):
#     plt.figure(figsize=(10, 6))
#     plt.plot(epoch_range, loss_log, label='Loss')
#     plt.plot(epoch_range, combined_loss_log, label='Combined Loss')
#     plt.title(title)
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

# def plot_fairness_components(epoch_range, fairness_penalty_sp_log, fairness_penalty_eo_log, fairness_penalty_approx_log, fairness_penalty_approx_eo_log, title):
#     plt.figure(figsize=(10, 6))
#     plt.plot(epoch_range, fairness_penalty_sp_log, label='Fairness Penalty SP')
#     plt.plot(epoch_range, fairness_penalty_eo_log, label='Fairness Penalty EO')
#     plt.plot(epoch_range, fairness_penalty_approx_log, label='Fairness Penalty Approx')
#     plt.plot(epoch_range, fairness_penalty_approx_eo_log, label='Fairness Penalty Approx EO')
#     plt.title(title)
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.show()

In [19]:
# def calculate_average_per_epoch(values, batches_per_epoch):
#     return [sum(values[i*batches_per_epoch:(i+1)*batches_per_epoch])/batches_per_epoch for i in range(num_epochs)]


In [20]:
# # Compute average values per epoch for each log
# teacher_loss_avg = calculate_average_per_epoch(teacher_loss_log, 500)
# teacher_combined_loss_avg = calculate_average_per_epoch(teacher_combined_loss_log, 500)
# teacher_fairness_penalty_sp_avg = calculate_average_per_epoch(teacher_fairness_penalty_sp_log, 500)
# teacher_fairness_penalty_eo_avg = calculate_average_per_epoch(teacher_fairness_penalty_eo_log, 500)
# teacher_fairness_penalty_approx_avg = calculate_average_per_epoch(teacher_fairness_penalty_approx_log, 500)
# teacher_fairness_penalty_approx_eo_avg = calculate_average_per_epoch(teacher_fairness_penalty_approx_eo_log, 500)

# student_loss_avg = calculate_average_per_epoch(student_loss_log, 500)
# student_combined_loss_avg = calculate_average_per_epoch(student_combined_loss_log, 500)
# student_fairness_penalty_sp_avg = calculate_average_per_epoch(student_fairness_penalty_sp_log, 500)
# student_fairness_penalty_eo_avg = calculate_average_per_epoch(student_fairness_penalty_eo_log, 500)
# student_fairness_penalty_approx_avg = calculate_average_per_epoch(student_fairness_penalty_approx_log, 500)
# student_fairness_penalty_approx_eo_avg = calculate_average_per_epoch(student_fairness_penalty_approx_eo_log, 500)


In [21]:
# # Assuming each loss log has one entry per epoch
# epoch_range = np.arange(num_epochs)

# # Plot for Teacher
# plot_loss_components(epoch_range, teacher_loss_avg, teacher_combined_loss_avg, 'Teacher Loss Components')
# plot_fairness_components(epoch_range, teacher_fairness_penalty_sp_avg, teacher_fairness_penalty_eo_avg, teacher_fairness_penalty_approx_avg, teacher_fairness_penalty_approx_eo_avg, 'Teacher Fairness Components')

# # Plot for Student
# plot_loss_components(epoch_range, student_loss_avg, student_combined_loss_avg, 'Student Loss Components')
# plot_fairness_components(epoch_range, student_fairness_penalty_sp_avg, student_fairness_penalty_eo_avg, student_fairness_penalty_approx_avg, student_fairness_penalty_approx_eo_avg, 'Student Fairness Components')


In [22]:
# Get metrics for both teacher and student
metrics_teacher = evaluate(teacher, testloader)
metrics_student = evaluate(student, testloader)
print('Teacher Metrics:', metrics_teacher)
print('Student Metrics:', metrics_student)

Teacher Metrics: (1.4128773212432861, 0.36201477080041955, 0.3391, 0.3391, 0.3287979733028897)
Student Metrics: (1.2786328792572021, 0.3042427291053151, 0.2816, 0.2816, 0.2681833395352643)


In [23]:
# Save the trained student model architecture and weights
torch.save(student.state_dict(), 'student_weights.pth')
torch.save(student.state_dict(), 'teacher_weights.pth')

# Save the model architecture to a file
with open('student_architecture.json', 'w') as f:
    f.write(str(student))

In [24]:
# # Create a new instance of ModifiableStudent
# num_new_classes = 52
# new_student = ModifiableStudent(num_classes=num_new_classes)

# # Load the state dictionary from the saved model file
# original_student_state_dict = torch.load('student_model.pth', map_location='cpu')

# # Filter out the fc2 layer from the original state dictionary and load it into the new_student instance
# new_student.load_state_dict({k: v for k, v in original_student_state_dict.items() if 'fc2' not in k}, strict=False)

# # Save the new model to a TorchScript .pt file
# new_student_scripted = torch.jit.script(new_student)
# new_student_scripted.save("new_student_model.pt")

In [25]:
# # Gather your metrics
# general_metrics = ['Precision', 'Recall', 'F1', 'Accuracy']
# teacher_general_metrics = list(metrics_teacher[1:])
# student_general_metrics = list(metrics_student[1:])

# x = np.arange(len(general_metrics))  # the label locations
# width = 0.35  # the width of the bars

# fig, axs = plt.subplots(4, 1, figsize=(10, 20))

# # Plot general metrics
# ax = axs[0]
# rects1 = ax.bar(x - width/2, teacher_general_metrics, width, label='Teacher')
# rects2 = ax.bar(x + width/2, student_general_metrics, width, label='Student')
# ax.set_xlabel('Metrics')
# ax.set_title('General Metrics by model and metric type')
# ax.set_xticks(x)
# ax.set_xticklabels(general_metrics)
# ax.legend()

# # Plot Inference Time
# ax = axs[1]
# rects1 = ax.bar(0 - width/2, metrics_teacher[0], width, label='Teacher')
# rects2 = ax.bar(0 + width/2, metrics_student[0], width, label='Student')
# ax.set_title('Inference Time by model')
# ax.set_ylabel('Time (seconds)')
# ax.set_xticks([0])
# ax.set_xticklabels(['Inference Time'])
# ax.legend()

# # Plot Number of Parameters
# ax = axs[2]
# rects1 = ax.bar(0 - width/2, teacher_num_parameters, width, label='Teacher')
# rects2 = ax.bar(0 + width/2, student_num_parameters, width, label='Student')
# ax.set_title('Number of Parameters by model')
# ax.set_ylabel('Number of Parameters')
# ax.set_xticks([0])
# ax.set_xticklabels(['Number of Parameters'])
# ax.legend()

# # Plot Model Size
# ax = axs[3]
# rects1 = ax.bar(0 - width/2, teacher_model_size, width, label='Teacher')
# rects2 = ax.bar(0 + width/2, student_model_size, width, label='Student')
# ax.set_title('Model Size by model')
# ax.set_ylabel('Size (bytes)')
# ax.set_xticks([0])
# ax.set_xticklabels(['Model Size'])
# ax.legend()

# fig.tight_layout()
# plt.show()